<a href="https://colab.research.google.com/github/rickvelloso/Learning-IA/blob/main/test_autoML_tunning_with_cancer_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [32]:
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

In [33]:
df_X = pd.DataFrame(X, columns=cancer.feature_names)
df_X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=12)

In [35]:
scaler = StandardScaler()
X_trained_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [36]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVN' : SVC(),
    'KNN' : KNeighborsClassifier(),
    'Naive Bayes' : GaussianNB(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
}



In [37]:
results = {}
for name, model in models.items():
    scores = cross_val_score(model, X_trained_scaled, y_train, cv=5)
    results[name] = scores.mean()

In [38]:
for name, score in results.items():
  print(f'{name}: {score:.4f}')

Logistic Regression: 0.9758
Decision Tree: 0.9319
Random Forest: 0.9604
SVN: 0.9692
KNN: 0.9648
Naive Bayes: 0.9385
Gradient Boosting: 0.9626
AdaBoost: 0.9758


In [39]:
best_model_name = max(results, key=results.get)
best_model = models[best_model_name]
print(best_model)

LogisticRegression()


In [40]:
best_model.fit(X_trained_scaled, y_train)

LogisticRegression()

In [41]:
y_pred = best_model.predict(X_test_scaled)

In [42]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9737


In [43]:
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[45  3]
 [ 0 66]]


In [44]:
cl_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(cl_report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        48
           1       0.96      1.00      0.98        66

    accuracy                           0.97       114
   macro avg       0.98      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



O resultado nos gerou um resultado interessante porém, para a situação (prever tumores) há um certo perigo com o recall de 0.94 para falsos positivos (casos aonde o tumor era maligno mas foi previsto como benigno).
Vou tentar corrigir isso, mesmo que perca um pouco de performance nos falsos negativos, porém são diagnosticos menos danosos.

In [45]:
y_probabilidades = best_model.predict_proba(X_test_scaled)[:, 1]

Para corrigir o problema anterior vou colocar um Threshold mais rigoroso, exigindo 99% de certeza para definir como benigno ou não.

In [47]:
NOVO_LIMIAR = 0.99
y_pred_novo_limiar = np.where(y_probabilidades > NOVO_LIMIAR, 1, 0)

In [48]:
print(f"\n--- Análise com Limiar Otimizado ({NOVO_LIMIAR}) ---")
cm_novo = confusion_matrix(y_test, y_pred_novo_limiar)
print("Nova Matriz de Confusão:")
print(cm_novo)


--- Análise com Limiar Otimizado (0.99) ---
Nova Matriz de Confusão:
[[47  1]
 [14 52]]


In [49]:
cl_report_novo = classification_report(y_test, y_pred_novo_limiar)
print("\nNovo Relatório de Classificação:")
print(cl_report_novo)


Novo Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.77      0.98      0.86        48
           1       0.98      0.79      0.87        66

    accuracy                           0.87       114
   macro avg       0.88      0.88      0.87       114
weighted avg       0.89      0.87      0.87       114



Apesar da redução de 2 casos de falso positivo, o aumento de 14 falsos negativos foi algo péssimo, esse novo teste com a limiar em 0.99 não teve reais vantagens em relação ao outro.
Agora vou tentar buscar um limiar perfeito, caso não consiga com esse modelo (logistic regression) tentarei com outro até buscar o 100% de recall para 0.

In [52]:
probabilidades_dos_erros_C01 = y_probabilidades[(y_test == 0) & (y_pred == 1)]
pior_erro_prob = probabilidades_dos_erros_C01.max()
LIMIAR_PERFEITO = pior_erro_prob + 0.001

In [55]:
print(f"Pior erro de probabilidade (C_01): {pior_erro_prob:.4f}")
print(f"Novo Limiar Perfeito (data-driven): {LIMIAR_PERFEITO:.4f}")

Pior erro de probabilidade (C_01): 0.9987
Novo Limiar Perfeito (data-driven): 0.9997


In [57]:
y_pred_limiar_perfeito = np.where(y_probabilidades > LIMIAR_PERFEITO, 1, 0)

In [58]:
print(f"\n--- Análise com Limiar Perfeito ---")
cm_perfeito = confusion_matrix(y_test, y_pred_limiar_perfeito)
print("Nova Matriz de Confusão:")
print(cm_perfeito)

cl_report_perfeito = classification_report(y_test, y_pred_limiar_perfeito)
print("\nNovo Relatório de Classificação:")
print(cl_report_perfeito)


--- Análise com Limiar Perfeito ---
Nova Matriz de Confusão:
[[48  0]
 [36 30]]

Novo Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.57      1.00      0.73        48
           1       1.00      0.45      0.62        66

    accuracy                           0.68       114
   macro avg       0.79      0.73      0.68       114
weighted avg       0.82      0.68      0.67       114



Os falsos positivos zeraram, porém para isso fizemos o recall de dos falsos negativos irem de 100% para 45%. O custo ainda está muito alto, acredito que o primeiro exemplo ainda possuia o melhor tradeoff.
Vou buscar agora por um modelo mais adequado.
Para isso, irei atras do segundo modelo mais forte encontrado no dicionario, la nas primerias celulas (adaboost e também vou testar o 3 colocado, svc).

In [59]:
model_ada = AdaBoostClassifier(random_state=12)
model_ada.fit(X_trained_scaled, y_train)

AdaBoostClassifier(random_state=12)

In [60]:
y_pred_ada = model_ada.predict(X_test_scaled)
cm_ada = confusion_matrix(y_test, y_pred_ada)
cl_report_ada = classification_report(y_test, y_pred_ada)

In [61]:
print("Matriz de Confusão (AdaBoost):")
print(cm_ada)
print("\nRelatório de Classificação (AdaBoost):")
print(cl_report_ada)

Matriz de Confusão (AdaBoost):
[[43  5]
 [ 1 65]]

Relatório de Classificação (AdaBoost):
              precision    recall  f1-score   support

           0       0.98      0.90      0.93        48
           1       0.93      0.98      0.96        66

    accuracy                           0.95       114
   macro avg       0.95      0.94      0.95       114
weighted avg       0.95      0.95      0.95       114



In [62]:
model_svc = SVC(random_state=12)
model_svc.fit(X_trained_scaled, y_train)

SVC(random_state=12)

In [63]:
y_pred_svc = model_svc.predict(X_test_scaled)
cm_svc = confusion_matrix(y_test, y_pred_svc)
cl_report_svc = classification_report(y_test, y_pred_svc)

In [64]:
print("Matriz de Confusão (SVC):")
print(cm_svc)
print("\nRelatório de Classificação (SVC):")
print(cl_report_svc)

Matriz de Confusão (SVC):
[[42  6]
 [ 0 66]]

Relatório de Classificação (SVC):
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        48
           1       0.92      1.00      0.96        66

    accuracy                           0.95       114
   macro avg       0.96      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114



In [67]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.metrics import make_scorer, recall_score

In [68]:
recall_class_0_scorer = make_scorer(recall_score, pos_label=0)

In [69]:
params_lr = {
    'C': np.logspace(-3, 3, 7),
    'solver': ['liblinear']
}

In [70]:
params_svc = {
    'C': np.logspace(-3, 3, 7),
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

In [71]:
params_ada = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0]
}

In [72]:
halving_lr = HalvingRandomSearchCV(LogisticRegression(max_iter=1000, random_state=12),
                                   params_lr, scoring=recall_class_0_scorer, cv=5, factor=3, random_state=12)

halving_svc = HalvingRandomSearchCV(SVC(random_state=12),
                                    params_svc, scoring=recall_class_0_scorer, cv=5, factor=3, random_state=12)

halving_ada = HalvingRandomSearchCV(AdaBoostClassifier(random_state=12),
                                    params_ada, scoring=recall_class_0_scorer, cv=5, factor=3, random_state=12)

In [73]:
print("\nBuscando pelo melhor LogisticRegression...")
halving_lr.fit(X_trained_scaled, y_train)

print("\nBuscando pelo melhor SVC...")
halving_svc.fit(X_trained_scaled, y_train)

print("\nBuscando pelo melhor AdaBoost...")
halving_ada.fit(X_trained_scaled, y_train)


Buscando pelo melhor LogisticRegression...


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 7 is smaller than n_iter=22. Running 7 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



Buscando pelo melhor SVC...

Buscando pelo melhor AdaBoost...


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 9 is smaller than n_iter=22. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


HalvingRandomSearchCV(estimator=AdaBoostClassifier(random_state=12),
                      param_distributions={'learning_rate': [0.01, 0.1, 1.0],
                                           'n_estimators': [50, 100, 200]},
                      random_state=12,
                      scoring=make_scorer(recall_score, response_method='predict', pos_label=0))

In [74]:
print("\n--- Resultados Finais no Teste ---")


--- Resultados Finais no Teste ---


In [75]:
y_pred_lr_auto = halving_lr.best_estimator_.predict(X_test_scaled)
print("\nMelhor LogisticRegression (Otimizado para Recall):")
print(f"Melhor pontuação (Recall C_0) no treino: {halving_lr.best_score_:.4f}")
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_lr_auto))


Melhor LogisticRegression (Otimizado para Recall):
Melhor pontuação (Recall C_0) no treino: 0.9600
Matriz de Confusão:
[[44  4]
 [ 1 65]]


In [76]:
y_pred_svc_auto = halving_svc.best_estimator_.predict(X_test_scaled)
print("\nMelhor SVC (Otimizado para Recall):")
print(f"Melhor pontuação (Recall C_0) no treino: {halving_svc.best_score_:.4f}")
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_svc_auto))


Melhor SVC (Otimizado para Recall):
Melhor pontuação (Recall C_0) no treino: 0.9442
Matriz de Confusão:
[[44  4]
 [ 0 66]]


In [77]:
y_pred_ada_auto = halving_ada.best_estimator_.predict(X_test_scaled)
print("\nMelhor AdaBoost (Otimizado para Recall):")
print(f"Melhor pontuação (Recall C_0) no treino: {halving_ada.best_score_:.4f}")
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_ada_auto))


Melhor AdaBoost (Otimizado para Recall):
Melhor pontuação (Recall C_0) no treino: 0.8983
Matriz de Confusão:
[[43  5]
 [ 0 66]]


Analisando os resultados, mesmo depois de varios processos, tanto de tuning quanto de autoML (um autoML limitado, por questões de custo, mas enfim), o resultado com melhor tradeoff ainda aparenta ser o primeiro. Apesar de 3 casos do pior erro possivel, que é o falso positivo, o recall em falsos negativos era completo.

Comparando os dois modelos mais extremos, o melhor e mais preciso ele funciona como um "assistente de confirmação", porém ainda com uma atenção para casos malignos, aonde ele pode falhar em detectar (o que é EXTREMAMENTE PERIGOSO). Daí, caberia ao médico utilizar somente para confirmar suspeitas de casos óbvios, mas jamais para dar alta.

Já o modelo com menor acurácia, mas recall perfeito (para falsos positivos) é uma ferramenta de triagem, ele caça bem os tumores malignos e é 100% eficaz nisso (no exemplo que estamos seguindo, claro.)
a partir daí o médico pode usar isso para dar altas, uma vez que o modelo não diz ser positivo sem ter total certeza de que é.